In [ ]:
"""Take biowulf produced temporal generalization files (1 for each training time)
and stitch them into single files for each participant"""
"""Note: This script relies on having generated a numpy file of testing timesfor each training time, script to do this
is in 'optional' and was run on a biowulf cluster"""


import numpy as np
import os
import glob

def load_subject_trainTime_data(subject_dir):
    "Loads in the individual numpy files for decoding at a specified training time"
    # Pattern to match all trainTime files
    file_pattern = os.path.join(subject_dir, "trainTime_*.npy")
    files = sorted(glob.glob(file_pattern))  # Sort by filename to order them properly by training time
    
    print(f"Found {len(files)} files for {subject_dir}")
    
    data_list = []
    for f in files:
        arr = np.load(f)
        data_list.append(arr)
    
    # Stack along a new axis
    # Old shape: (n_runs, n_test_times), stacked shape: (n_runs, n_train_times, n_test_times)
    data = np.stack(data_list, axis=1)
    print(f"Loaded data shape: {data.shape}")
    return data



subjects = [f"S{i:02}" for i in range(1, 21)]  #Select subjects to analyse
base_dir = '/volumes/OrientationDecoding/bids_dirOld-ignoreJeff/derivatives/WithinStill' #Set Directory
save_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/!Important Data/TempGenWithinStill/data'

all_subjects_data = []

for subj in subjects:
    subj_dir = os.path.join(base_dir, subj)
    subj_data = load_subject_trainTime_data(subj_dir)
    all_subjects_data.append(subj_data)
    save_path = os.path.join(save_dir, f'{subj}_trainTime_data.npy')
    np.save(save_path, subj_data)
    print(f"Saved {subj} data to {save_path}")

#Stack subjects: shape (n_subjects, train_times, test_times)
all_subjects_data = np.stack(all_subjects_data, axis=0)
stacked_path = os.path.join(save_dir, 'allSubjects_trainTime_data.npy')
np.save(stacked_path, all_subjects_data)
print(f"All subjects combined data shape: {all_subjects_data.shape}")



Found 961 files for /volumes/OrientationDecoding/bids_dirOld-ignoreJeff/derivatives/WithinStill/S01


KeyboardInterrupt: 

In [2]:
import matplotlib.pyplot as plt
import numpy as np

# all_subjects_data: shape (subjects, train_times, test_times)
mean_accuracy = np.mean(all_subjects_data, axis=0)  # shape: (train_times, test_times)

plt.figure(figsize=(8,6))
im = plt.imshow(mean_accuracy, 
                origin='lower',   # so training time=0 is at bottom
                aspect='auto', 
                interpolation='nearest', 
                cmap='viridis')

plt.colorbar(im, label='Decoding Accuracy')
plt.xlabel('Testing Time Index')
plt.ylabel('Training Time Index')
plt.title('Mean Decoding Accuracy: Training time (Y) vs Testing time (X)')
plt.tight_layout()
plt.show()


NameError: name 'all_subjects_data' is not defined